In [1]:
!pip install pandas

In [2]:
!pip install pyarrow

In [3]:
!pip install scikit-learn

In [4]:
import pandas as pd

In [5]:
from io import BytesIO

In [6]:
import pyarrow.parquet as pq

df = pq.read_table(source='~/Downloads/yellow_tripdata_2023-01.parquet').to_pandas()
# df = pd.read_parquet(, engine='pyarrow')

In [7]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [8]:
df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60

In [9]:
df[df.tpep_dropoff_datetime < df.tpep_pickup_datetime]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
1839180,1,2023-01-20 13:35:00,2023-01-20 13:05:48,1.0,0.8,99.0,N,227,14,1,16.5,0.0,0.5,0.0,0.0,1.0,18.0,0.0,0.0,-29.200000
2124414,1,2023-01-23 10:43:58,2023-01-23 10:29:26,1.0,2.4,99.0,N,102,82,1,21.5,0.0,0.5,0.0,0.0,1.0,23.0,0.0,0.0,-14.533333
2218504,1,2023-01-24 11:00:00,2023-01-24 10:40:48,1.0,0.4,99.0,N,247,169,1,15.5,0.0,0.5,0.0,0.0,1.0,17.0,0.0,0.0,-19.200000


In [10]:
pd.set_option('display.float_format', '{:.2f}'.format)
df.duration.describe()

count   3066766.00
mean         15.67
std          42.59
min         -29.20
25%           7.12
50%          11.52
75%          18.30
max       10029.18
Name: duration, dtype: float64

In [11]:
df_oliers_removed = df[(df.duration<=60) & (df.duration>=1)]

In [12]:
df_oliers_removed.shape[0]/df.shape[0]

0.9812202822125979

In [13]:
df_use = df_oliers_removed[['duration', 'PULocationID', 'DOLocationID']]

In [14]:
from sklearn.feature_extraction import DictVectorizer

In [15]:
# # Step 1: Convert the pickup and dropoff location IDs to strings (to avoid label encoding)
# df_use['PULocationID'] = df_use['PULocationID'].astype(str)
# df_use['DOLocationID'] = df_use['DOLocationID'].astype(str)

# # Step 2: Turn the DataFrame into a list of dictionaries, excluding the non-vectorized column
# data_dicts = df_use[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# # Step 3: Fit the DictVectorizer and transform the data (only for the location ID columns)
# vectorizer = DictVectorizer(sparse=False)  # Set sparse=False if you want a dense matrix
# feature_matrix = vectorizer.fit_transform(data_dicts)

# # Step 4: Add the non-vectorized columns (e.g., 'other_column') to the feature matrix
# other_column = df_use['duration'].values.reshape(-1, 1)

# # Concatenate the feature matrix with the non-vectorized column
# import numpy as np
# final_feature_matrix = np.concatenate([feature_matrix, other_column], axis=1)

# # Step 5: Get the feature names (including the new column names)
# feature_names = list(vectorizer.get_feature_names_out()) + ['duration']

# # Display the results
# print("Feature Matrix (dense form):")
# print(final_feature_matrix)

# Convert location IDs to strings for clearer dummy column names
df_use['PULocationID'] = df_use['PULocationID'].astype(str)
df_use['DOLocationID'] = df_use['DOLocationID'].astype(str)

# Apply one-hot encoding separately to pickup and dropoff location IDs
pu_dummies = pd.get_dummies(df_use['PULocationID'], prefix='PU')
do_dummies = pd.get_dummies(df_use['DOLocationID'], prefix='DO')

# Combine original 'duration' with the encoded features
df_encoded = pd.concat([df_use[['duration']], pu_dummies, do_dummies], axis=1)

# Show the final encoded DataFrame
print(df_encoded.shape)

/tmp/ipykernel_5775/1553809303.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['PULocationID'] = df_use['PULocationID'].astype(str)
/tmp/ipykernel_5775/1553809303.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use['DOLocationID'] = df_use['DOLocationID'].astype(str)


(3009173, 516)


: 

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

X = df_encoded.loc[:df_encoded.shape[0]/10].drop('duration', axis=1)
y = df_encoded.loc[:df_encoded.shape[0]/10]['duration']

# Train linear regression model
model = SGDRegressor(max_iter=1000, tol=1e-3)
model.fit(X, y)


In [ ]:

# Predict on training data
y_pred = model.predict(X)

# Calculate RMSE
rmse = mean_squared_error(y, y_pred, squared=False)
print(f"RMSE on train: {rmse:.3f}")